## PROJECT BDDA 2
### PART 1 INTRODUCTION
### UMANG DHAR DWIVEDI 015017
##### In the following file we use the data set of covid sentiments and find the accuracy of model ( During or trail we found out that the data set has to be broken down and used in fraction portion for successful run of model.)
##### The following project has been done on virtual machine 
##### 1. START HADOOP USING allstart.sh
##### 2. START SPARK USING pysparknb
##### 3. Copy the data using hadoop fs -copyFromLocal filename

In [ ]:
import gc # garbage collector has been used to overcome the problems faced during trail run. error such as low memory space.

In [ ]:
gc.collect()

152

In [ ]:
from pyspark.ml import Pipeline 
from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import col, udf,regexp_replace,isnull
from pyspark.sql.types import StringType,IntegerType
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import length

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Bass').getOrCreate()

In [ ]:
df=spark.read.csv("Corona_NLP_train.csv", sep=",", header=True, inferSchema=True)

In [ ]:
df7= df.sample(fraction=0.7)

####  Taking a sample (70%) because the size of the file is too large to perform modelling.

In [ ]:
df7.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|            UserName|          ScreenName|            Location|             TweetAt|       OriginalTweet|         Sentiment|
+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|                3800|               48752|                  UK|          16-03-2020|advice Talk to yo...|          Positive|
|                3801|               48753|           Vagabonds|          16-03-2020|Coronavirus Austr...|          Positive|
|              PLEASE|         don't panic| THERE WILL BE EN...|                null|                null|              null|
|           Stay calm|          stay safe.|                null|                null|                null|              null|
|                3803|               48755|                null|          16-03-2020|Me, ready to go a...|            

In [ ]:
df7.count()

47758

In [ ]:
df7.columns #to get the header(title of the coulmns)

['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']

In [ ]:
df7.printSchema() #to understand the coulmns attributes,their type if null values permited or not

root
 |-- UserName: string (nullable = true)
 |-- ScreenName: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- TweetAt: string (nullable = true)
 |-- OriginalTweet: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [ ]:
df7.describe().show() ## used to see the statistical values of the data

+-------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+
|summary|            UserName|          ScreenName|            Location|     TweetAt|       OriginalTweet|           Sentiment|
+-------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+
|  count|               47755|               39026|               24023|       29287|               29046|               20018|
|   mean| 9.594871878485131E7|  184885.56355580172|2.600001766788235...|        10.0|               682.0|                null|
| stddev|  9.40864829357664E9|1.9643792152831886E7|1.072007417131545...|         NaN|  1176.0629234866644|                null|
|    min|                   ?|                    |                    |            |      Coronavirus...| "" Well covid-19...|
|    max|Ø  South-based au...|          but for us|ï? ???????'? ????...| she says."|Ø  As buyers stoc..

In [ ]:
df7['sentiment'] ##lets understand SENTIMENT coulmn

Column<b'sentiment'>

In [ ]:
type(df7['sentiment']) ##rdd helps in converting data in form of rows and coulmns

pyspark.sql.column.Column

In [ ]:
df7.select('sentiment').show()

+------------------+
|         sentiment|
+------------------+
|          Positive|
|          Positive|
|              null|
|              null|
|              null|
|              null|
|              null|
|          Positive|
|          Negative|
|           Neutral|
|              null|
|              null|
|Extremely Positive|
|              null|
|              null|
|              null|
|              null|
|              null|
|              null|
|              null|
+------------------+
only showing top 20 rows



In [ ]:
type(df7.select('sentiment')) 

pyspark.sql.dataframe.DataFrame

In [ ]:
df7.head(2) ##spark stores data in form of rows and coulmns not in form 

[Row(UserName='3800', ScreenName='48752', Location='UK', TweetAt='16-03-2020', OriginalTweet='advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order', Sentiment='Positive'),
 Row(UserName='3801', ScreenName='48753', Location='Vagabonds', TweetAt='16-03-2020', OriginalTweet='Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P', Sentiment='Positive')]

In [ ]:
df7.head(30)[4] ## 30 used to take the number of rows and in that we want to view value of row that is numbered 4

Row(UserName='3803', ScreenName='48755', Location=None, TweetAt='16-03-2020', OriginalTweet='Me, ready to go at supermarket during the #COVID19 outbreak.', Sentiment=None)

In [ ]:
df.select(['Location','TweetAt','Sentiment']) ##now lets see the values if we only need 3 coulmn

DataFrame[Location: string, TweetAt: string, Sentiment: string]

In [ ]:
df.select(['Location','TweetAt','Sentiment']).show()

+--------------------+--------------------+---------+
|            Location|             TweetAt|Sentiment|
+--------------------+--------------------+---------+
|              London|          16-03-2020|  Neutral|
|                  UK|          16-03-2020| Positive|
|           Vagabonds|          16-03-2020| Positive|
|                null|          16-03-2020|     null|
| THERE WILL BE EN...|                null|     null|
|                null|                null|     null|
|                null|                null|     null|
|                null|          16-03-2020|     null|
|          but please| don't panic. It ...|     null|
|                null|                null|     null|
|ÜT: 36.319708,-82...|          16-03-2020| Positive|
|35.926541,-78.753267|          16-03-2020| Positive|
|             Austria|          16-03-2020|     null|
|                null|                null|     null|
|     Atlanta, GA USA|          16-03-2020| Positive|
|    BHAVNAGAR,GUJRAT|      

In [ ]:
df7.groupBy("Location").count().show() ## to see how population is distributed in various LOCATIONS

+--------------------+-----+
|            Location|count|
+--------------------+-----+
| Brisbane, Australia|    8|
|          South Asia|    1|
|West Woofle-Dust ...|    1|
|   St Petersburg, FL|   11|
| All across Michigan|    1|
|     Northumberland |    1|
|   Dallas, Texas USA|    1|
|some where around...|    1|
|           Worcester|    3|
| Mumbai, Maharashtra|    3|
|           Bangalore|   21|
|           Norn Iron|    1|
|Horsham, Pennsylv...|    1|
|            Novi, MI|    1|
|Sagaon, Kalyan Do...|    1|
|       Shimla  India|    1|
|Ferrara, Emilia R...|    1|
|      Luton, England|    3|
|              Heaven|    1|
|         black site |    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
df7.groupby("Location").count().orderBy("count", ascending=False).show() #finding the higest count from a particular location

+--------------------+-----+
|            Location|count|
+--------------------+-----+
|                null|23735|
|              London|  378|
|       United States|  375|
|     London, England|  358|
|        New York, NY|  277|
|      Washington, DC|  264|
|      United Kingdom|  242|
|     Los Angeles, CA|  196|
|               India|  183|
|                 USA|  157|
|                  UK|  154|
|           Australia|  152|
|            Positive|  144|
|              Canada|  138|
|            Negative|  126|
|England, United K...|  125|
|         Chicago, IL|  122|
|              Global|  113|
|       New York, USA|  106|
|  Extremely Negative|  105|
+--------------------+-----+
only showing top 20 rows



#### We select  the columns that are needed for prediction as tweet and its sentiment

In [ ]:
category = df7.select("OriginalTweet", "Sentiment")

In [ ]:
category.show()

+--------------------+------------------+
|       OriginalTweet|         Sentiment|
+--------------------+------------------+
|advice Talk to yo...|          Positive|
|Coronavirus Austr...|          Positive|
|                null|              null|
|                null|              null|
|Me, ready to go a...|              null|
|                null|              null|
|                null|              null|
|Due to COVID-19 o...|          Positive|
|For corona preven...|          Negative|
|All month there h...|           Neutral|
|Due to the Covid-...|              null|
|                null|              null|
|#horningsea is a ...|Extremely Positive|
|Me: I don't need ...|              null|
|                null|              null|
|                null|              null|
|????? ????? ?????...|              null|
|                null|              null|
|                null|              null|
|"@eyeonthearctic ...|              null|
+--------------------+------------

#### We will Check for NULL values in Sentiment and Tweet

In [ ]:
def null_value_count(df7):
    null_columns_counts = []
    numRows = df7.count()
    for k in df7.columns:
        nullRows = df7.where(col(k).isNull()).count()
        if(nullRows > 0):
            temp = k,nullRows
            null_columns_counts.append(temp)
            return(null_columns_counts)



In [ ]:
null_columns_count_list = null_value_count(category)

In [ ]:
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|         OriginalTweet|            18712|
+----------------------+-----------------+



#### Dropping the null values

In [ ]:
category = category.dropna()

In [ ]:
category.show()

+--------------------+------------------+
|       OriginalTweet|         Sentiment|
+--------------------+------------------+
|advice Talk to yo...|          Positive|
|Coronavirus Austr...|          Positive|
|Due to COVID-19 o...|          Positive|
|For corona preven...|          Negative|
|All month there h...|           Neutral|
|#horningsea is a ...|Extremely Positive|
|For those who are...|          Positive|
|with 100  nations...|Extremely Negative|
|In preparation fo...|          Negative|
|This morning I te...|Extremely Negative|
|There Is of in th...|          Negative|
|Went to the super...|           Neutral|
|Worried about the...|          Positive|
|my wife works ret...|          Negative|
|Now I can go to t...|          Positive|
|CHECK VIDEO ?? ht...|Extremely Negative|
|Breaking Story: O...|           Neutral|
|This is the line ...|           Neutral|
| Please Share  Kn...|Extremely Positive|
|"""Everything we...|            racism|
+--------------------+------------

In [ ]:
category.count()


20018

####  Let's Split text into words

In [ ]:
regex_tokenizer = RegexTokenizer(inputCol="OriginalTweet", outputCol="words", pattern="\\W")
raw_words = regex_tokenizer.transform(category)

In [ ]:
raw_words.show()

+--------------------+------------------+--------------------+
|       OriginalTweet|         Sentiment|               words|
+--------------------+------------------+--------------------+
|advice Talk to yo...|          Positive|[advice, talk, to...|
|Coronavirus Austr...|          Positive|[coronavirus, aus...|
|Due to COVID-19 o...|          Positive|[due, to, covid, ...|
|For corona preven...|          Negative|[for, corona, pre...|
|All month there h...|           Neutral|[all, month, ther...|
|#horningsea is a ...|Extremely Positive|[horningsea, is, ...|
|For those who are...|          Positive|[for, those, who,...|
|with 100  nations...|Extremely Negative|[with, 100, natio...|
|In preparation fo...|          Negative|[in, preparation,...|
|This morning I te...|Extremely Negative|[this, morning, i...|
|There Is of in th...|          Negative|[there, is, of, i...|
|Went to the super...|           Neutral|[went, to, the, s...|
|Worried about the...|          Positive|[worried, abou

Removing Stopwords

In [ ]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df7 = remover.transform(raw_words)

In [ ]:
words_df7.select("words","filtered").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                                                                                                                                                                                                                                                                                                                      |filtered                                          

#### Now we will Encode the Sentiment column

In [ ]:
indexer = StringIndexer(inputCol="Sentiment", outputCol="SentimentIndex")
feature_data = indexer.fit(words_df7).transform(words_df7)

In [ ]:
feature_data.select("Sentiment","SentimentIndex").show()

+------------------+--------------+
|         Sentiment|SentimentIndex|
+------------------+--------------+
|          Positive|           0.0|
|          Positive|           0.0|
|          Positive|           0.0|
|          Negative|           1.0|
|           Neutral|           2.0|
|Extremely Positive|           3.0|
|          Positive|           0.0|
|Extremely Negative|           4.0|
|          Negative|           1.0|
|Extremely Negative|           4.0|
|          Negative|           1.0|
|           Neutral|           2.0|
|          Positive|           0.0|
|          Negative|           1.0|
|          Positive|           0.0|
|Extremely Negative|           4.0|
|           Neutral|           2.0|
|           Neutral|           2.0|
|Extremely Positive|           3.0|
|            racism|         281.0|
+------------------+--------------+
only showing top 20 rows





#### We will Convert text into vectors of token counts.


In [ ]:
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(feature_data)
countVectorizer_feateures = model.transform(feature_data)

In [ ]:
(trainingData, testData) = countVectorizer_feateures.randomSplit([0.8, 0.2])

In [ ]:
nb = NaiveBayes(modelType="multinomial",labelCol="SentimentIndex", featuresCol="features")
nbModel = nb.fit(trainingData)
nb_predictions = nbModel.transform(testData)

In [ ]:
nb_predictions.select("prediction", "SentimentIndex", "features").show(30)

+----------+--------------+--------------------+
|prediction|SentimentIndex|            features|
+----------+--------------+--------------------+
|       1.0|           1.0|(41541,[18,68,94,...|
|       3.0|           0.0|(41541,[19,72,111...|
|       1.0|           4.0|(41541,[8,16,21,3...|
|       4.0|           0.0|(41541,[54,153,39...|
|       0.0|           2.0|(41541,[3,53,54,5...|
|       3.0|           3.0|(41541,[8,21,26,2...|
|       4.0|           1.0|(41541,[68,236,59...|
|       0.0|           3.0|(41541,[3,4,6,19,...|
|       0.0|           3.0|(41541,[11,13,15,...|
|       0.0|           4.0|(41541,[10,12,21,...|
|       0.0|           2.0|(41541,[6,95,118,...|
|       0.0|           1.0|(41541,[3,4,8,11,...|
|       4.0|           4.0|(41541,[2,10,31,7...|
|       1.0|           4.0|(41541,[5,11,22,4...|
|       1.0|           1.0|(41541,[7,9,279,2...|
|       1.0|           0.0|(41541,[5,36,40,6...|
|       0.0|           0.0|(41541,[16,17,150...|
|       0.0|        

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="SentimentIndex", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))
print("Test Error of NaiveBayes = %g " % (1.0 - nb_accuracy))

Accuracy of NaiveBayes is = 0.444051
Test Error of NaiveBayes = 0.555949 


### The accuracy comes out to be 44% and test error as 55%

##**THANKS** 